<!--# How to write a custom dataset class-->
# 커스텀 데이터셋 클래스를 만드는 방법

<!--A typical statagy to improve generalization in deep neural networks is to increase the number of training examples which allows more parameters to be used in the model. Even if the number of parameters is kept unchanged, increasing the number of training examples often improves generalization performance.-->
deep neural network에서 일반화 성능을 높이는 전형적인 방법은 학습 데이터의 수를 늘려서 model로 하여금 파라미터를 더 잘 수정하게끔 하는 것입니다. 파라미터의 수가 바뀌지않고도 학습 데이터의 수를 늘리는 것만으로도 일반화 성능을 높여주곤 하거든요.

<!--Since it can be tedious and expensive to manually obtain and label additional training examples, a useful stratagy is to consider methods for automatically increasing the size of a training set. Fortunately, for image datasets, there are several augmentation methods that have been found to work well in practice. They include:-->
추가적인 학습 데이터를 만들어주는것은 수동으로 하기엔 귀찮고 번거로운 일이 아닐 수 없습니다. 따라서 가장 현명한 방법은 자동으로 학습 데이터셋을 늘려주는 방법을 구상하는 것이겠죠. 다행히도 이미지 데이터셋에는 여러 가지 증강 방법이 제시되어 있기 때문에 실제로 사용해볼 때 유용합니다. 아래와 같은 방법이 포함되어있습니다 :

<!--* Randomly cropping several slightly smaller images from the original training image.-->
* 원본 학습 이미지에서 임의의 지점을 잘라서 작은 이미지를 만듭니다.
<!--* Horizontally flipping the image.-->
* 수평을 기준으로 이미지를 뒤집습니다.
<!--* Randomly rotating the image.-->
* 임의의 각도로 이미지를 돌립니다.
<!--* Applying various distortions and or noise to the images, etc.-->
* 다양한 왜곡 혹은 잡음을 넣어 이미지를 변형합니다. 기타등등..

<!--In this example, we will write a custom dataset class that performs the first two of these augmentation methods on the CIFAR10 dataset. We will then train our previous deep CNN and check that the generalization performance on the test set has in fact improved.-->
이 예제에서는 CIFAR10 데이터셋에서 2가지의 증강 방법을 사용하여 커스텀 데이터셋을 만들어 볼 것입니다. 이후 이것을 가지고 학습한 model과 이전의 CNN model을 비교하여 성능이 개선이 되었는지 직접 확인해 볼 것입니다.

<!--We will create this dataset augmentation class as a subclass of `DatasetMixin`, which has the following API:-->
`DatasetMixin` 이라는 데이터셋 증강 클래스의 서브클래스로써 만들어봅시다. 이 클래스는 아래와 같은 인터페이스를 갖습니다 :

<!--- `__len\__` method to return the size of data in dataset. -->
- `__len__` 메서드는 데이터셋의 크기를 반환합니다.
<!--- `get_example` method to return data or a tuple of data and label, which are passed by 'i' argement variable. -->
- `get_example` 메서드는 i 인수에 의해 데이터 튜플과 라벨을 반환합니다.

<!--Other necesary features for a dataset can be prepared by inheriting 'chainer.datase.DasetMixin' class. -->
데이터셋의 다른 필수요소들은 'chainer.dataset.DatasetMixin' 클래스를 상속받는 것 만으로도 설정할 수 있습니다.

<!--## 1. Write the dataset augmentation class for CIFAR10-->
## 1. CIFAR10을 기반으로 한 증강 데이터셋 클래스 작성하기

In [1]:
import numpy as np
from chainer import dataset
from chainer.datasets import cifar

class CIFAR10Augmented(dataset.DatasetMixin):

    def __init__(self, train=True):
        train_data, test_data = cifar.get_cifar10()
        if train:
            self.data = train_data
        else:
            self.data = test_data
        self.train = train
        self.random_crop = 4

    def __len__(self):
        return len(self.data)

    def get_example(self, i):
        x, t = self.data[i]
        if self.train:
            x = x.transpose(1, 2, 0)
            h, w, _ = x.shape
            x_offset = np.random.randint(self.random_crop)
            y_offset = np.random.randint(self.random_crop)
            x = x[y_offset:y_offset + h - self.random_crop,
                  x_offset:x_offset + w - self.random_crop]
            if np.random.rand() > 0.5:
                x = np.fliplr(x)
            x = x.transpose(2, 0, 1)
        return x, t

<!--This class performs the following types of data augmentation on the CIFAR10 example images:-->
이 클래스는 아래와 같은 데이터 증강 방법을 통해 CIFAR10 데이터셋의 데이터를 증가시키고 있습니다 :

<!--- Randomly crop a 28X28 area form the 32X32 whole image data.-->
- 32x32 공간에서 임의로 28x28 공간을 자릅니다.
<!--- Randomly perform a horizontal flip with 0.5 probability. -->
- 매번 확률을 계산하여 50% 이상일 경우 수평방향으로 이미지를 뒤집습니다.

<!--## 2. Train on the CIFAR10 dataset using our dataset augmentation class-->
## 2. 커스텀 데이터셋을 통해 학습하기
<!--Let's now train the same deep CNN from the previous example. The only difference is that we will now use our dataset augmentation class. Since we reuse the same model with the same number of parameters, we can observe how much the augmentation improves the test set generalization performance.-->
이제 이전에 작성했던 깊은 DNN을 학습시켜봅시다. 이전 예제와 다른 점이라 한다면 여기에서는 데이터셋 증강 클래스를 사용해서 학습을 시킬 것이라는 것이죠. 이전 model의 파라미터 또한 그대로 사용하지만 일반화 성능이 증강 데이터셋에서 얼마나 증가하는지 확인해볼 수 있습니다.

In [2]:
import chainer
import chainer.functions as F
import chainer.links as L
from chainer.datasets import cifar
from chainer import iterators
from chainer import optimizers
from chainer import training
from chainer.training import extensions
'''
현재 이 코드는 GPU를 사용하며 기본 gpu id를 0으로 설정해두고 학습을 수행하고 있습니다.
GPU를 이용한 학습을 하지 않으시려면 아래 train 함수에서 표기된 부분을 변경하세요.
(model, updater, evaluator)
'''
class ConvBlock(chainer.Chain):
    
    def __init__(self, n_ch, pool_drop=False):
        w = chainer.initializers.HeNormal()
        '''
        # Chainer v1
        super(ConvBlock, self).__init__(
            conv=L.Convolution2D(None, n_ch, 3, 1, 1,
                                 nobias=True, initialW=w),
            bn=L.BatchNormalization(n_ch)
        )
        
        self.train = True
        self.pool_drop = pool_drop
        '''
        # Chainer v2
        super(ConvBlock, self).__init__()
        with self.init_scope():
            self.conv = L.Convolution2D(None, n_ch, 3, 1, 1,
                                       nobias=True, initialW=w)
            self.bn = L.BatchNormalization(n_ch)
            self.pool_drop = pool_drop
        
    def __call__(self, x):
        h = F.relu(self.bn(self.conv(x)))
        if self.pool_drop:
            h = F.max_pooling_2d(h, 2, 2)
            h = F.dropout(h, ratio=0.25)
        return h
    
class LinearBlock(chainer.Chain):
    
    def __init__(self):
        w = chainer.initializers.HeNormal()
        '''
        # Chainer v1
        super(LinearBlock, self).__init__(
            fc=L.Linear(None, 1024, initialW=w))
        self.train = True
        '''
        # Chainer v2
        super(LinearBlock, self).__init__()
        with self.init_scope():
            self.fc = L.Linear(None, 1024, initialW=w)
        
    def __call__(self, x):
        return F.dropout(F.relu(self.fc(x)), ratio=0.5)
    
class DeepCNN(chainer.ChainList):

    def __init__(self, n_output):
        super(DeepCNN, self).__init__(
            ConvBlock(64),
            ConvBlock(64, True),
            ConvBlock(128),
            ConvBlock(128, True),
            ConvBlock(256),
            ConvBlock(256),
            ConvBlock(256),
            ConvBlock(256, True),
            LinearBlock(),
            LinearBlock(),
            L.Linear(None, n_output)
        )
        self._train = True
            
    @property
    def train(self):
        return self._train
            
    @train.setter
    def train(self, val):
        self._train = val
        for c in self.children():
            c.train = val
    
    def __call__(self, x):
        for f in self.children():
            x = f(x)
        return x

def train(model_object, batchsize=64, gpu_id=0, max_epoch=20):

    # 1. Dataset
    train, test = CIFAR10Augmented(), CIFAR10Augmented(False)

    # 2. Iterator
    train_iter = iterators.SerialIterator(train, batchsize)
    test_iter = iterators.SerialIterator(test, batchsize, False, False)

    # 3. Model
    model = L.Classifier(model_object)
    model.to_gpu(gpu_id) # GPU 사용

    # 4. Optimizer
    optimizer = optimizers.Adam()
    optimizer.setup(model)

    # 5. Updater
    updater = training.StandardUpdater(train_iter, optimizer, device=gpu_id) # GPU 사용

    # 6. Trainer
    trainer = training.Trainer(updater, (max_epoch, 'epoch'), out='{}_cifar10augmented_result'.format(model_object.__class__.__name__))

    # 7. Evaluator

    class TestModeEvaluator(extensions.Evaluator):

        def evaluate(self):
            model = self.get_target('main')
            model.train = False
            ret = super(TestModeEvaluator, self).evaluate()
            model.train = True
            return ret

    trainer.extend(extensions.LogReport())
    trainer.extend(TestModeEvaluator(test_iter, model, device=gpu_id)) # GPU 사용
    trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'main/accuracy', 'validation/main/loss', 'validation/main/accuracy', 'elapsed_time']))
    trainer.extend(extensions.PlotReport(['main/loss', 'validation/main/loss'], x_key='epoch', file_name='loss.png'))
    trainer.extend(extensions.PlotReport(['main/accuracy', 'validation/main/accuracy'], x_key='epoch', file_name='accuracy.png'))
    # trainer.extend(extensions.ProgressBar(None, 1000))
    trainer.run()
    del trainer
    
    return model
    
model = train(DeepCNN(10), max_epoch=100)

epoch       main/loss   main/accuracy  validation/main/loss  validation/main/accuracy  elapsed_time
1           2.05572     0.2291         1.74695               0.340665                  17.4928       
2           1.5648      0.404109       1.34217               0.493531                  34.2186       
3           1.30648     0.510563       1.18946               0.54787                   51.0198       
4           1.13766     0.59387        0.990061              0.663117                  67.7972       
5           0.978814    0.671016       0.833926              0.721835                  84.5843       
6           0.869395    0.713128       0.748523              0.752389                  101.349       
7           0.762867    0.748119       0.707316              0.770104                  118.147       
8           0.706639    0.771907       0.897042              0.761047                  134.912       
9           0.648668    0.79186        0.573436              0.811107               

79          0.100458    0.967229       0.495916              0.909833                  1324.93       
80          0.103642    0.966989       0.527921              0.90824                   1341.72       
81          0.107007    0.965633       0.507009              0.908041                  1358.47       
82          0.102158    0.96833        0.551649              0.903762                  1375.22       
83          0.104669    0.967049       0.592808              0.905255                  1391.99       
84          0.106911    0.967009       0.546619              0.899781                  1408.7        
85          0.10126     0.96853        0.526428              0.909136                  1425.43       
86          0.104149    0.96741        0.46494               0.910529                  1442.14       
87          0.0992295   0.96903        0.435181              0.907444                  1458.85       
88          0.0950871   0.970591       0.516829              0.905653             

일반화 정확도가 개선된 것이 보이시나요? 이전 model과 비교해봅시다.